# 네이버지식인 크롤링

site url: https://kin.naver.com/search/list.naver?query=%EB%B0%94%EC%9D%B4%EB%B8%8C%EC%BD%94%EB%94%A9

crawling 결과
- 제목
- 링크
- 날짜
- 카테고리
- 답변수

## selector에 특수 문자가 있는 경우
`class="txt_g1 _nclicks:kin.cat1"`가 있다고 가정하자.  
여기서 알맞게 select를 하려면 `특수문자를 escape` 처리해야 한다.  
이 때 `class indicator`인 맨 앞에 존재하는 `.`문자는 배제해야 한다.

_**AS-IS**_  
`"txt_g1 _nclicks:kin.cat1"`

_**TO-BE**_
1. 개발자 도구: .txt\_g1.\_nclicks\:kin\.cat1
   👉🏻 `\`하나만 사용하여 escape 처리
2. python: .txt\\_g1.\\_nclicks\\:kin\\.cat1
   👉🏻 `\`를 두 개인 `\\`를 사용하여 escape 처리



# 하나의 지식인만 크롤링


In [79]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlencode

ROOT = "https://kin.naver.com"
query="바이브코딩"
PATH = "search/list.naver"

def get_kin_url(root, path,query):
  base_url = urljoin(root, path)
  query_string = urlencode({"query": query})

  return f"{base_url}?{query_string}"

url = get_kin_url(ROOT, PATH, query)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
one_kin_card = soup.select_one(".basic1 > li:nth-of-type(2)")

title = one_kin_card.select_one("._searchListTitleAnchor").text
link = one_kin_card.select_one("._searchListTitleAnchor").get("href")
date = one_kin_card.select_one(".txt_inline").text[:-1]
category = one_kin_card.select_one("._nclicks\\:kin\\.cat2").text
answer_count = one_kin_card.select_one(".hit").text.split(" ")[-1]

print(title, link, date, category, answer_count, sep="\n")

바이브코딩 공부 필요없나요?
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=104&docId=485315883&qb=67CU7J2067iM7L2U65Sp&enc=utf8
2025.08.06
프로그래밍
10


# 하나의 페이지에 존재하는 지식인 크롤링


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlencode

ROOT = "https://kin.naver.com"
query="바이브코딩"
PATH = "search/list.naver"

def get_kin_url(root, path,query):
  base_url = urljoin(root, path)
  query_string = urlencode({"query": query})

  return f"{base_url}?{query_string}"

url = get_kin_url(ROOT, PATH, query)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
kins = soup.select(".basic1 li")

print("한 페이지에 있는 지식인 개수", len(kins))
print("-"*100)

for kin in kins:
  title = kin.select_one("._searchListTitleAnchor").text
  link = kin.select_one("._searchListTitleAnchor").get("href")
  date = kin.select_one(".txt_inline").text[:-1]
  category = kin.select_one("._nclicks\\:kin\\.cat2").text
  answer_count = kin.select_one(".hit").text.split(" ")[-1]

  print(title, link, date, category, answer_count, sep="\n")
  print("="*100)



한 페이지에 있는 지식인 개수 10
----------------------------------------------------------------------------------------------------
작성완료했는데, 바이브 코딩으로... 
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10401&docId=489809572&qb=67CU7J2067iM7L2U65Sp&enc=utf8
2025.10.28
애플리케이션프로그래밍
2
바이브코딩 공부 필요없나요?
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=104&docId=485315883&qb=67CU7J2067iM7L2U65Sp&enc=utf8
2025.08.06
프로그래밍
10
바이브코딩과 css, Java script
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1040202&docId=485525994&qb=67CU7J2067iM7L2U65Sp&enc=utf8
2025.07.01
자바스크립트
4
바이브코딩 시대에도
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10405&docId=486253999&qb=67CU7J2067iM7L2U65Sp&enc=utf8
2025.07.23
프로그래머
10
커서 ai(바이브 코딩)플랫폼 제작 1대1... 
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10504&docId=489600457&qb=67CU7J2067iM7L2U65Sp&enc=utf8
2025.10.20
웹마스터
4
바이브 코딩 Replit AI 앱을 만들었는대..
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1040104&docId=486273502&qb=67CU7J2067iM7L2U65Sp&enc=utf8
202

# 여러 페이지에 존재하는 지식인 크롤링(Pagination)

In [91]:
from typing import Optional, TypedDict
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlencode

ROOT = "https://kin.naver.com"
query="바이브코딩"
PATH = "search/list.naver"

class URLParams(TypedDict):
  root: str
  path: str
  query: Optional[str]
  page: Optional[int]

def get_url(url_params: URLParams):
  base_url = urljoin(url_params["root"], url_params["path"])
  query_string = urlencode({
    "query": url_params["query"] or "바이브코딩", 
    "page": url_params["page"] or 1
  })

  return f"{base_url}?{query_string}"

def get_kins_per_page(soup: BeautifulSoup):
  result = []

  for kin in soup:
    title = kin.select_one("._searchListTitleAnchor").text
    link = kin.select_one("._searchListTitleAnchor").get("href")
    date = kin.select_one(".txt_inline").text[:-1]
    category = kin.select_one("._nclicks\\:kin\\.cat2").text
    answer_count = kin.select_one(".hit").text.split(" ")[-1]

    result.append({
      "title": title,
      "link": link,
      "date": date,
      "category": category,
      "answer_count": answer_count
    })

  return result

def get_all_kins(root: str, path: str,query: str, page_count: int):
  result = []

  for page in range(1, page_count + 1):
    url = get_url({
      "root": root,
      "path": path,
      "query": query,
      "page": page
    })
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    kins = soup.select(".basic1 li")

    result.extend(get_kins_per_page(kins))
  
  return result
    
page_count = int(input("크롤링할 페이지 개수 입력:"))
result = get_all_kins(ROOT, PATH, query, page_count)

print("총 크롤링한 지식인 개수", len(result))
print("="*100)

result


총 크롤링한 지식인 개수 30


[{'title': '작성완료했는데, 바이브 코딩으로... ',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10401&docId=489809572&qb=67CU7J2067iM7L2U65Sp&enc=utf8',
  'date': '2025.10.28',
  'category': '애플리케이션프로그래밍',
  'answer_count': '2'},
 {'title': '바이브코딩 공부 필요없나요?',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=104&docId=485315883&qb=67CU7J2067iM7L2U65Sp&enc=utf8',
  'date': '2025.08.06',
  'category': '프로그래밍',
  'answer_count': '10'},
 {'title': '바이브코딩과 css, Java script',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1040202&docId=485525994&qb=67CU7J2067iM7L2U65Sp&enc=utf8',
  'date': '2025.07.01',
  'category': '자바스크립트',
  'answer_count': '4'},
 {'title': '바이브코딩 시대에도',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10405&docId=486253999&qb=67CU7J2067iM7L2U65Sp&enc=utf8',
  'date': '2025.07.23',
  'category': '프로그래머',
  'answer_count': '10'},
 {'title': '커서 ai(바이브 코딩)플랫폼 제작 1대1... ',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1

# 

# 마지막 페이지 처리
`바이브 코딩`으로 검색했을 때 마지막 페이지는 6 페이지에서 끝난다.  
다음이라는 navigation도 없으면 page에 따른 content가 없는데도 page_number는 존재한다.

즉, 10번 page를 누르면 그냥 `검색 결과 없음`이라고 뜬다.  
👉🏻 해결책: 한 페이지에 crawling한 결과가 `10개`이므로 10개 보다 작다면 crawling을 중단한다.

In [101]:
from typing import Optional, TypedDict
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlencode

ROOT = "https://kin.naver.com"
query="바이브코딩"
PATH = "search/list.naver"

class URLParams(TypedDict):
  root: str
  path: str
  query: Optional[str]
  page: Optional[int]

def get_url(url_params: URLParams):
  base_url = urljoin(url_params["root"], url_params["path"])
  query_string = urlencode({
    "query": url_params["query"] or "바이브코딩", 
    "page": url_params["page"] or 1,
    "section": "kin"
  })

  return f"{base_url}?{query_string}"

def get_kins_per_page(soup: BeautifulSoup):
  result = []

  for kin in soup:
    title = kin.select_one("._searchListTitleAnchor").text
    link = kin.select_one("._searchListTitleAnchor").get("href")
    date = kin.select_one(".txt_inline").text[:-1]
    category = kin.select_one("._nclicks\\:kin\\.cat2").text
    answer_count = kin.select_one(".hit").text.split(" ")[-1]

    result.append({
      "title": title,
      "link": link,
      "date": date,
      "category": category,
      "answer_count": answer_count
    })

  return result

def get_all_kins(root: str, path: str,query: str, page_count: int):
  result = []

  for page in range(1, page_count + 1):
    url = get_url({
      "root": root,
      "path": path,
      "query": query,
      "page": page
    })
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    kins = soup.select(".basic1 li")

    kins_per_page = get_kins_per_page(kins)
    result.extend(kins_per_page)

    print(len(kins_per_page))
    if len(kins_per_page) < 10: # 한 페이지에 존재하는 지식인 개수
      break
  
  return result
    
page_count = int(input("크롤링할 페이지 개수 입력:"))
result = get_all_kins(ROOT, PATH, query, page_count)

result


10
10
10
10
10
8


[{'title': '작성완료했는데, 바이브 코딩으로... ',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10401&docId=489809572&qb=67CU7J2067iM7L2U65Sp&enc=utf8',
  'date': '2025.10.28',
  'category': '애플리케이션프로그래밍',
  'answer_count': '2'},
 {'title': '바이브코딩 공부 필요없나요?',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=104&docId=485315883&qb=67CU7J2067iM7L2U65Sp&enc=utf8',
  'date': '2025.08.06',
  'category': '프로그래밍',
  'answer_count': '10'},
 {'title': '바이브코딩과 css, Java script',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1040202&docId=485525994&qb=67CU7J2067iM7L2U65Sp&enc=utf8',
  'date': '2025.07.01',
  'category': '자바스크립트',
  'answer_count': '4'},
 {'title': '바이브코딩 시대에도',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10405&docId=486253999&qb=67CU7J2067iM7L2U65Sp&enc=utf8',
  'date': '2025.07.23',
  'category': '프로그래머',
  'answer_count': '10'},
 {'title': '커서 ai(바이브 코딩)플랫폼 제작 1대1... ',
  'link': 'https://kin.naver.com/qna/detail.naver?d1id=1